In [ ]:
# pro Programm:
# - Zyklen entfernen, bei denen Farben aufeinander folgen, die per Definition nicht aufeinander folgen dürfen
# - Zyklen entfernen, die deutlich länger oder kürzer (+/- 50%) als der Median aller Zyklen in dem Programm sind
# 
# Zählen, wie oft pro Thing Zyklen entfernt werden# Prepare import of modules from parent directory.

import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import json



with open('observations_example_timerange_reconstructed_cycles.json', 'r') as fp:
    observations_example_timerange_reconstructed_cycles = json.load(fp)